In [ ]:
import numpy as np
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import polars as pl
from sklearn.metrics import mean_squared_error, mean_absolute_error
from joblib import Parallel, delayed
from more_itertools import chunked
from functools import reduce
from typing import List
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import os


El objetivo de este Notebook es preparar los datos para modelar con redes neuronales

In [ ]:
# Reducir memoria automáticamente
def optimizar_memoria(df):
    for col in df.select_dtypes(include=['int64', 'int32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['float64', 'float32']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    return df

In [ ]:
# Abrir el archivo parquet y cargarlo en un DataFrame data/l_vm_completa_train_pendientes.parquet
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_train.parquet', engine='fastparquet')

In [ ]:
# Eliminar de df_agrupado las filas donde la columna A_PREDECIR sea 'N'
df_full = df_full[df_full['A_PREDECIR'] != 'N']
df_full = df_full.drop(columns=['A_PREDECIR'])

In [ ]:
df_agrupado = (
    df_full.groupby([
        'ORDINAL','PERIODO', 'ANIO', 'MES', 'MES_SIN', 'MES_COS', 'TRIMESTRE',
        'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'PRODUCT_ID'
    ], as_index=False)[['TN', 'CLASE']].sum()
)

In [ ]:
del df_full
gc.collect()

In [ ]:
# Mostrar las filas de df_agrupado ordernadas por TN de menor a mayor
df_agrupado = df_agrupado.sort_values(by='TN', ascending=True)
# Mostrar las primeras 10 filas de df_agrupado
print(df_agrupado.head(10))

In [ ]:
# Agregar una columna que indique la diferencia en ORDINAL entre el ORDINAL actual y el ORDINAL anterior donde TN sea mayor a 0
# para ese PRODUCT_ID

from joblib import Parallel, delayed
import numpy as np
import pandas as pd

def calcular_mejoras_por_grupo(grupo):
    grupo = grupo.sort_values('ORDINAL').copy()
    ult_ordinal = None
    valores = []

    for _, row in grupo.iterrows():
        if ult_ordinal is None:
            valores.append(36)
        else:
            valores.append(int(row['ORDINAL'] - ult_ordinal))

        if row['TN'] > 0:
            ult_ordinal = row['ORDINAL']

    grupo['MESES_SIN_COMPRAR_PRODUCT_ID'] = np.array(valores, dtype=np.int16)
    return grupo

def agregar_diferencia_ordinal_parallel(df: pd.DataFrame, n_jobs: int = -1) -> pd.DataFrame:
    df = df.copy()
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = 36  # valor inicial
    df['MESES_SIN_COMPRAR_PRODUCT_ID'] = df['MESES_SIN_COMPRAR_PRODUCT_ID'].astype('int16')

    # Agrupar por cliente y producto
    grupos = list(df.groupby(['PRODUCT_ID']))

    # Procesar en paralelo
    resultados = Parallel(n_jobs=n_jobs, backend='loky', batch_size=128)(
        delayed(calcular_mejoras_por_grupo)(grupo) for _, grupo in grupos
    )

    # Concatenar todos los resultados
    df_resultado = pd.concat(resultados, axis=0).sort_index()
    return df_resultado



df_agrupado = agregar_diferencia_ordinal_parallel(df_agrupado, n_jobs=28)

In [ ]:
# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si PERIODO es 201906 o 201908 o 201910, y 0 en caso contrario
# Calcular los días del mes usando las columnas ANIO y MES

# Agregar a df_full una variable categorica MES_PROBLEMATICO que sea 1 si ANIO==2019 y MES en [6, 8, 10], y 0 en caso contrario
df_agrupado['MES_PROBLEMATICO'] = np.where(
       (df_agrupado['ANIO'] == 2019) & (df_agrupado['MES'].isin([6, 8, 10])),
       1., 0.0
)
df_agrupado['MES_PROBLEMATICO'] = df_agrupado['MES_PROBLEMATICO'].astype(np.float32)

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from joblib import Parallel, delayed

def calcular_pendientes_grupo(group, periodos_list):
    group = group.sort_values(by='PERIODO').copy()
    n = len(group)
    y_values = group['TN'].values
    for cant in periodos_list:
        means = np.zeros(n)
        pendientes = np.zeros(n)
        ewmas = np.zeros(n)
        medians = np.zeros(n)
        minimo = np.zeros(n)
        maximo = np.zeros(n)
        stds = np.zeros(n)
        skews = np.zeros(n)
        kurts = np.zeros(n)
        growths = np.zeros(n)
        iqrs = np.zeros(n)
        sums = np.zeros(n)
        count_pos = np.zeros(n)
        pct_zero = np.zeros(n)
        last = np.zeros(n)
        last_diff = np.zeros(n)

        coef_var = np.zeros(n)
        maxmin_ratio = np.zeros(n)
        rango = np.zeros(n)
        rango_rel = np.zeros(n)
        last_vs_median = np.zeros(n)
        cambio_ventana = np.zeros(n)
        zeros_end = np.zeros(n)
        last_pct_sum = np.zeros(n)
        pct_90 = np.zeros(n)
        pct_10 = np.zeros(n)
        pct_width = np.zeros(n)

        # --- Comparaciones TN vs rolling stats ---
        tn_minus_mean = np.zeros(n)
        tn_minus_median = np.zeros(n)
        tn_minus_ewma = np.zeros(n)
        tn_over_mean = np.zeros(n)
        tn_over_median = np.zeros(n)
        tn_over_ewma = np.zeros(n)

        x = np.arange(cant)
        for i in range(n):
            # Solo procesar si la ventana está completa
            if i >= cant - 1:
                y = y_values[i - (cant - 1): i + 1]
                means[i] = np.mean(y)
                try:
                    pendientes[i] = np.polyfit(x, y, 1)[0]
                except:
                    pendientes[i] = 0
                ewmas[i] = pd.Series(y).ewm(span=cant, adjust=False).mean().iloc[-1]
                medians[i] = np.median(y)
                minimo[i] = np.min(y)
                maximo[i] = np.max(y)
                std_y = np.std(y)
                mean_y = means[i]
                stds[i] = std_y
                skews[i] = skew(y) if std_y > 0 else 0
                kurts[i] = kurtosis(y) if std_y > 0 else 0
                growths[i] = (y[-1] - y[0]) / (y[0] + 1e-8) if y[0] != 0 else 0
                iqrs[i] = np.percentile(y, 75) - np.percentile(y, 25)
                sums[i] = np.sum(y)
                count_pos[i] = np.sum(y > 0)
                pct_zero[i] = np.mean(y == 0)
                last[i] = y[-1]
                last_diff[i] = y[-1] - y[-2] if cant > 1 else 0

                coef_var[i] = std_y / (mean_y + 1e-8) if mean_y != 0 else 0
                maxmin_ratio[i] = maximo[i] / (minimo[i] + 1e-8) if minimo[i] != 0 else 0
                rango[i] = maximo[i] - minimo[i]
                rango_rel[i] = rango[i] / (minimo[i] + 1e-8) if minimo[i] != 0 else 0
                last_vs_median[i] = y[-1] - np.median(y)
                # Cambio de ventana anterior a actual
                if i >= 2 * cant - 1:
                    y_prev = y_values[i - (2 * cant - 1):i - cant + 1]
                    mean_prev = np.mean(y_prev)
                    cambio_ventana[i] = (mean_y - mean_prev) / (mean_prev + 1e-8) if mean_prev != 0 else 0
                zeros_end[i] = np.argmax(y[::-1] != 0)
                last_pct_sum[i] = y[-1] / (np.sum(y) + 1e-8) if np.sum(y) != 0 else 0
                pct_90[i] = np.percentile(y, 90)
                pct_10[i] = np.percentile(y, 10)
                pct_width[i] = pct_90[i] - pct_10[i]

                # Comparaciones TN vs rolling stats
                tn_actual = y_values[i]
                tn_minus_mean[i] = tn_actual - means[i]
                tn_minus_median[i] = tn_actual - medians[i]
                tn_minus_ewma[i] = tn_actual - ewmas[i]
                tn_over_mean[i] = tn_actual / (means[i] + 1e-8) if means[i] != 0 else 0
                tn_over_median[i] = tn_actual / (medians[i] + 1e-8) if medians[i] != 0 else 0
                tn_over_ewma[i] = tn_actual / (ewmas[i] + 1e-8) if ewmas[i] != 0 else 0
            # Si la ventana no está completa, todo queda en cero (ya inicializado)

        group[f'TN_MEAN_{str(cant).zfill(2)}'] = means
        group[f'PENDIENTE_TENDENCIA_{cant}'] = pendientes
        group[f'TN_EWMA_{str(cant).zfill(2)}'] = ewmas
        group[f'TN_MEDIAN_{str(cant).zfill(2)}'] = medians
        group[f'TN_MIN_{str(cant).zfill(2)}'] = minimo
        group[f'TN_MAX_{str(cant).zfill(2)}'] = maximo
        group[f'TN_STD_{str(cant).zfill(2)}'] = stds
        group[f'TN_SKEW_{str(cant).zfill(2)}'] = skews
        group[f'TN_KURT_{str(cant).zfill(2)}'] = kurts
        group[f'TN_GROWTH_{str(cant).zfill(2)}'] = growths
        group[f'TN_IQR_{str(cant).zfill(2)}'] = iqrs
        group[f'TN_SUM_{str(cant).zfill(2)}'] = sums
        group[f'TN_COUNT_POS_{str(cant).zfill(2)}'] = count_pos
        group[f'TN_PCT_ZERO_{str(cant).zfill(2)}'] = pct_zero
        group[f'TN_LAST_{str(cant).zfill(2)}'] = last
        group[f'TN_LAST_DIFF_{str(cant).zfill(2)}'] = last_diff

        group[f'TN_COEF_VAR_{cant}'] = coef_var
        group[f'TN_MAXMIN_RATIO_{cant}'] = maxmin_ratio
        group[f'TN_RANGO_{cant}'] = rango
        group[f'TN_RANGO_REL_{cant}'] = rango_rel
        group[f'TN_LAST_VS_MEDIAN_{cant}'] = last_vs_median
        group[f'TN_CHANGE_PREV_WINDOW_{cant}'] = cambio_ventana
        group[f'TN_ZEROS_END_{cant}'] = zeros_end
        group[f'TN_LAST_PCT_SUM_{cant}'] = last_pct_sum
        group[f'TN_PCT90_{cant}'] = pct_90
        group[f'TN_PCT10_{cant}'] = pct_10
        group[f'TN_PCT_WIDTH_{cant}'] = pct_width

        # Comparaciones TN vs rolling stats
        group[f'TN_MINUS_MEAN_{str(cant).zfill(2)}'] = tn_minus_mean
        group[f'TN_MINUS_MEDIAN_{str(cant).zfill(2)}'] = tn_minus_median
        group[f'TN_MINUS_EWMA_{str(cant).zfill(2)}'] = tn_minus_ewma
        group[f'TN_OVER_MEAN_{str(cant).zfill(2)}'] = tn_over_mean
        group[f'TN_OVER_MEDIAN_{str(cant).zfill(2)}'] = tn_over_median
        group[f'TN_OVER_EWMA_{str(cant).zfill(2)}'] = tn_over_ewma

    return group

def calcular_pendientes_parallel(df, periodos_list=[6, 12], n_jobs=20):
    group_cols = [col for col in df.columns if col != 'TN']
    print("Agrupando por columnas:", group_cols)
    grupos = [group for _, group in df.groupby(group_cols)]
    resultados = Parallel(n_jobs=n_jobs, backend='loky', verbose=10)(
        delayed(calcular_pendientes_grupo)(group, periodos_list) for group in grupos
    )
    df_final = pd.concat(resultados, ignore_index=True)
    df_final.replace([np.inf, -np.inf], 0, inplace=True)
    df_final.fillna(0, inplace=True)
    return df_final

# --- Script principal ---
# Calcular pendientes, medias ponderadas y medianas para los periodos indicados en paralelo
df_agrupado = calcular_pendientes_parallel(df_agrupado, periodos_list=[3, 6, 9, 12, 18], n_jobs=28)


In [ ]:
print(df_agrupado.columns.tolist())

In [ ]:
# Hacer un EDA de la columna  CLASE en  df_agrupado
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- Estadísticas descriptivas ---
print("Estadísticas básicas de CLASE:")
print(df_agrupado['CLASE'].describe())

print("\nPrimeros valores únicos (si son pocos):", df_agrupado['CLASE'].unique()[:10])
print("Cantidad de valores únicos:", df_agrupado['CLASE'].nunique())

print("\nCantidad de nulos:", df_agrupado['CLASE'].isna().sum())

# --- Histograma ---
plt.figure(figsize=(8,4))
sns.histplot(df_agrupado['CLASE'], bins=30, kde=True)
plt.title("Histograma de CLASE")
plt.xlabel("CLASE")
plt.ylabel("Frecuencia")
plt.show()

# --- Boxplot para ver outliers ---
plt.figure(figsize=(8,2))
sns.boxplot(x=df_agrupado['CLASE'])
plt.title("Boxplot de CLASE")
plt.show()

# --- Distribución log-transformada (útil si hay muchos valores chicos y pocos grandes) ---
plt.figure(figsize=(8,4))
sns.histplot(np.log1p(df_agrupado['CLASE']), bins=30, kde=True)
plt.title("Histograma de log(1+CLASE)")
plt.xlabel("log(1+CLASE)")
plt.ylabel("Frecuencia")
plt.show()

# --- Percentiles y cuantiles ---
print("\nPercentiles de CLASE:")
print(df_agrupado['CLASE'].quantile([0, 0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99, 1]))

# --- Outliers extremos (más allá de 3 desviaciones estándar) ---
media = df_agrupado['CLASE'].mean()
std = df_agrupado['CLASE'].std()
outliers = df_agrupado[(df_agrupado['CLASE'] > media + 3*std) | (df_agrupado['CLASE'] < media - 3*std)]
print(f"\nCantidad de outliers (>3 desvíos): {len(outliers)}")

# --- Relación con otras variables (ejemplo: PERIODO) ---
if 'PERIODO' in df_agrupado.columns:
    plt.figure(figsize=(10,4))
    sns.lineplot(x='PERIODO', y='CLASE', data=df_agrupado, ci=None)
    plt.title("CLASE a lo largo del tiempo (PERIODO)")
    plt.show()




In [ ]:
df_agrupado['CLASE_LOG1P'] = np.log1p(df_agrupado['CLASE'])
# Y para invertir:
#df_agrupado['CLASE'] = np.expm1(df_full['CLASE_LOG1P'])

# Eliminar la columna CLASE 
df_agrupado = df_agrupado.drop(columns=['CLASE'])


In [ ]:
from sklearn.preprocessing import StandardScaler

# FIT scaler solo sobre entrenamiento
mask_train = df_agrupado['PERIODO'] <= 201909
scaler_y = StandardScaler()
scaler_y.fit(df_agrupado.loc[mask_train, ['CLASE_LOG1P']])

# Aplicar la transformación a todo df_full
df_agrupado['CLASE_LOG1P_Z'] = scaler_y.transform(df_agrupado[['CLASE_LOG1P']])

# Guardar media y std
media_y = scaler_y.mean_[0]
std_y = scaler_y.scale_[0]
print(f"Media usada (entrenamiento): {media_y}")
print(f"Std usada (entrenamiento): {std_y}")


In [ ]:
import joblib

# Guardar a disco
joblib.dump(scaler_y, 'scaler_y_CLASE_LOG1P.joblib')


In [ ]:
import joblib

# Cargar desde disco
#scaler_y = joblib.load('scaler_y_CLASE_LOG1P.joblib')
#y_pred_log1p = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).flatten()



In [ ]:
# Eliminar las columnas CLASE y CLASE_LOG1P para evitar confusiones
df_agrupado.drop(columns=['CLASE', 'CLASE_LOG1P'], inplace=True, errors='ignore')

In [ ]:
print(df_agrupado.columns.tolist())

In [ ]:
# Guardar el DataFrame en parquet
df_agrupado.to_parquet('./data/product_interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [113]:
df_agrupado = pd.read_parquet('./data/product_interm_NN_TORCH.parquet', index=False, engine='fastparquet')

In [114]:
# Excluir la variable de clase
cols_to_check = df_agrupado.columns.difference(['CLASE_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_agrupado[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [115]:
print(f"📊 DataFrame final con {df_agrupado.shape[0]:,} filas y {df_agrupado.shape[1]} columnas:")

📊 DataFrame final con 22,375 filas y 182 columnas:


In [116]:
# Copia de seguridad del DataFrame
df = df_agrupado.copy()

# === Binning (en deciles) ===
df['PRODUCT_RANK_BIN'] = pd.qcut(df['PRODUCT_ID'], q=10, labels=False)
df['PRODUCT_RANK_BIN'] = df['PRODUCT_RANK_BIN'].astype('category')

# Reemplazar en el DataFrame principal
df_agrupado = df


In [117]:
cat_cols = ['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE','MES_PROBLEMATICO','PRODUCT_RANK_BIN']

for col in cat_cols:
    if df_agrupado[col].isnull().any():
        df_agrupado[col] = df_agrupado[col].fillna("missing")


encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    df_agrupado.loc[:, col] = le.fit_transform(df_agrupado[col]).astype(int)
    encoders[col] = le  # para guardar los mapeos por si necesitás revertirlos

# Guardo los encoders en archivos .pkl
import joblib
import os

os.makedirs('encoders', exist_ok=True)

for col, le in encoders.items():
    joblib.dump(le, f'encoders/{col}_encoder.pkl')

In [118]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import os

# Copia del DataFrame original
df = df_agrupado.copy()

# 1. Columnas categóricas (auto si no existen)
if 'cat_cols' not in locals():
    cat_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in ['PRODUCT_RANK_BIN']:
    if col not in cat_cols:
        cat_cols.append(col)

# 2. Columnas a excluir del escalado
excluir = ['PERIODO', 'CLASE_LOG1P_Z', 'MES_SIN', 'MES_COS'] + cat_cols
print("Columnas excluidas del escalado:", excluir)

# 3. Columnas numéricas a escalar
cols_a_escalar = [
    col for col in df.columns
    if col not in excluir and pd.api.types.is_numeric_dtype(df[col])
]
print("Columnas numéricas a escalar:", cols_a_escalar)


Columnas excluidas del escalado: ['PERIODO', 'CLASE_LOG1P_Z', 'MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'MES_PROBLEMATICO', 'PRODUCT_RANK_BIN']
Columnas numéricas a escalar: ['ORDINAL', 'ANIO', 'MES', 'TRIMESTRE', 'PRODUCT_ID', 'TN', 'MESES_SIN_COMPRAR_PRODUCT_ID', 'TN_MEAN_03', 'PENDIENTE_TENDENCIA_3', 'TN_EWMA_03', 'TN_MEDIAN_03', 'TN_MIN_03', 'TN_MAX_03', 'TN_STD_03', 'TN_SKEW_03', 'TN_KURT_03', 'TN_GROWTH_03', 'TN_IQR_03', 'TN_SUM_03', 'TN_COUNT_POS_03', 'TN_PCT_ZERO_03', 'TN_LAST_03', 'TN_LAST_DIFF_03', 'TN_COEF_VAR_3', 'TN_MAXMIN_RATIO_3', 'TN_RANGO_3', 'TN_RANGO_REL_3', 'TN_LAST_VS_MEDIAN_3', 'TN_CHANGE_PREV_WINDOW_3', 'TN_ZEROS_END_3', 'TN_LAST_PCT_SUM_3', 'TN_PCT90_3', 'TN_PCT10_3', 'TN_PCT_WIDTH_3', 'TN_MINUS_MEAN_03', 'TN_MINUS_MEDIAN_03', 'TN_MINUS_EWMA_03', 'TN_OVER_MEAN_03', 'TN_OVER_MEDIAN_03', 'TN_OVER_EWMA_03', 'TN_MEAN_06', 'PENDIENTE_TENDENCIA_6', 'TN_EWMA_06', 'TN_MEDIAN_06', 'TN_MIN_06', 'TN_MAX_06', 'TN_STD_06', 'TN_SKEW_06', '

In [119]:

# 4. Entrenamiento del scaler SOLO con datos de entrenamiento
df_entrenamiento = df[df['PERIODO'] <= 201910].copy()
scaler = StandardScaler()
scaler.fit(df_entrenamiento[cols_a_escalar])

# 5. Aplicación del scaler a TODO el dataset
valores_escalados = scaler.transform(df[cols_a_escalar])
df_scaled = pd.DataFrame(
    valores_escalados,
    columns=[col + '_Z' for col in cols_a_escalar],
    index=df.index
)

# 6. Eliminar columnas originales escaladas (excepto IDs)
cols_a_borrar = [col for col in cols_a_escalar if col not in ['CUSTOMER_ID', 'PRODUCT_ID']]
df.drop(columns=cols_a_borrar, inplace=True)

# 7. Concatenar columnas escaladas
df = pd.concat([df, df_scaled], axis=1)

# 8. Guardar el scaler
os.makedirs('./encoders', exist_ok=True)
joblib.dump(scaler, './encoders/scaler_features_numericas.pkl')
print("Scaler guardado en './encoders/scaler_features_numericas.pkl'")

# 9. Actualizar df_full
df_agrupado = df

print("Shape final del DataFrame:", df_agrupado.shape)
print("Primeras columnas escaladas:\n", df_agrupado[[col + '_Z' for col in cols_a_escalar]].head())



Scaler guardado en './encoders/scaler_features_numericas.pkl'
Shape final del DataFrame: (22375, 184)
Primeras columnas escaladas:
    ORDINAL_Z    ANIO_Z     MES_Z  TRIMESTRE_Z  PRODUCT_ID_Z      TN_Z  \
0  -1.819956 -1.299285 -1.613484    -1.339623      0.403997 -0.355391   
1  -1.819956 -1.299285 -1.613484    -1.339623     -0.449898 -0.311829   
2  -1.819956 -1.299285 -1.613484    -1.339623     -0.528071 -0.302478   
3  -1.819956 -1.299285 -1.613484    -1.339623     -0.952012 -0.106872   
4  -1.819956 -1.299285 -1.613484    -1.339623     -1.069272  0.042260   

   MESES_SIN_COMPRAR_PRODUCT_ID_Z  TN_MEAN_03_Z  PENDIENTE_TENDENCIA_3_Z  \
0                        5.067999           0.0                      0.0   
1                        5.067999           0.0                      0.0   
2                        5.067999           0.0                      0.0   
3                        5.067999           0.0                      0.0   
4                        5.067999           0.0  

In [120]:
print(df_agrupado.columns.tolist())

['PERIODO', 'MES_SIN', 'MES_COS', 'ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'PRODUCT_ID', 'MES_PROBLEMATICO', 'CLASE_LOG1P_Z', 'PRODUCT_RANK_BIN', 'ORDINAL_Z', 'ANIO_Z', 'MES_Z', 'TRIMESTRE_Z', 'PRODUCT_ID_Z', 'TN_Z', 'MESES_SIN_COMPRAR_PRODUCT_ID_Z', 'TN_MEAN_03_Z', 'PENDIENTE_TENDENCIA_3_Z', 'TN_EWMA_03_Z', 'TN_MEDIAN_03_Z', 'TN_MIN_03_Z', 'TN_MAX_03_Z', 'TN_STD_03_Z', 'TN_SKEW_03_Z', 'TN_KURT_03_Z', 'TN_GROWTH_03_Z', 'TN_IQR_03_Z', 'TN_SUM_03_Z', 'TN_COUNT_POS_03_Z', 'TN_PCT_ZERO_03_Z', 'TN_LAST_03_Z', 'TN_LAST_DIFF_03_Z', 'TN_COEF_VAR_3_Z', 'TN_MAXMIN_RATIO_3_Z', 'TN_RANGO_3_Z', 'TN_RANGO_REL_3_Z', 'TN_LAST_VS_MEDIAN_3_Z', 'TN_CHANGE_PREV_WINDOW_3_Z', 'TN_ZEROS_END_3_Z', 'TN_LAST_PCT_SUM_3_Z', 'TN_PCT90_3_Z', 'TN_PCT10_3_Z', 'TN_PCT_WIDTH_3_Z', 'TN_MINUS_MEAN_03_Z', 'TN_MINUS_MEDIAN_03_Z', 'TN_MINUS_EWMA_03_Z', 'TN_OVER_MEAN_03_Z', 'TN_OVER_MEDIAN_03_Z', 'TN_OVER_EWMA_03_Z', 'TN_MEAN_06_Z', 'PENDIENTE_TENDENCIA_6_Z', 'TN_EWMA_06_Z', 'TN_MEDIAN_06_Z', 'TN_MIN_06_Z', '

In [122]:
# Excluir la variable de clase
cols_to_check = df_agrupado.columns.difference(['CLASE_LOG1P_Z'])

# Calcular cantidad de NaNs por columna
nan_columns = df_agrupado[cols_to_check].isna().sum()

# Filtrar solo las columnas que tienen al menos un NaN
nan_columns = nan_columns[nan_columns > 0].sort_values(ascending=False)

# Mostrar
print(nan_columns)

Series([], dtype: int64)


In [124]:
print(df_agrupado.shape)
# Guardar el DataFrame resultante en un archivo parquet
df_agrupado.to_parquet('./data/product_train_val_NN_TORCH.parquet', engine='fastparquet', index=False)

(22375, 184)


In [125]:
print(cat_cols)

['ID_CAT1', 'ID_CAT2', 'ID_CAT3', 'ID_BRAND', 'SKU_SIZE', 'MES_PROBLEMATICO', 'PRODUCT_RANK_BIN']
